### Contents

- [Configurations and Libraries](#Configurations-and-Libraries)
- [Functions](#Functions)

### Header

In [1]:
# import libraries

# maths
import scipy.stats as stats
import numpy as np
import pandas as pd
from haversine import haversine, Unit

# others
from tqdm import tqdm
import time
import datetime as dt
import re
import os

# web
import json

In [2]:
# file paths

raw_path = '../../data/0_raw/fitrec/' 
input_path = '../../data/1_input/fitrec/'
clean_path = '../../data/2_clean/fitrec/' 
preprocess_path = '../../data/3_preprocess/fitrec/' 
output_path = '../../data/4_output/fitrec/' 

### Functions

In [3]:
# output smaller json file with incremental filenames e.g. xxx_123.json

def output_json(data,file_no):
    
    file_str = str(file_no)
    file_str = file_str.zfill(4)            
    out_path_2 = out_path + file_str + '.json'

    with open(out_path_2, 'w') as f:
        print(out_path_2)
        json.dump(data, f)

In [4]:
def compute_distance(record,idx):
    
    #print(record['latitude'])
    #print(record['longitude'])
    
    for i in range(1,500):
        
        cur_loc = (record['latitude'][i],record['longitude'][i])
        prev_loc = (record['latitude'][i-1],record['longitude'][i-1])
        
        # metre
        dist = haversine(cur_loc,prev_loc) * 1000     
        record['distance'][i] = dist
        
        #print(i,prev_loc,cur_loc,dist)
        
    return record

In [5]:
def compute_speed(record,idx):
    
    for i in range(1,500):
        
        # seconds
        time_diff = record['timestamp'][i] - record['timestamp'][i-1]
        record['time_diff'][i] = time_diff
         
        if time_diff > 0:
            
            # metre/seconds
            speed = record['distance'][i] / time_diff
            
            # km/h
            record['speed'][i] = speed * (60*60/1000)
            
        elif time_diff == 0:
            record['speed'][i] = record['speed'][i-1]
            #print('error: time_diff = 0 for {}:{}'.format(idx,i))
            
        # time_diff < 0
        else:
            record['speed'][i] = record['speed'][i-1]
            #print('error: time_diff < 0 for {}:{}'.format(idx,i))         
        
        #print(i,time_diff,speed)
        
    # set speed for index 0
#     for i in range(1,500):
#         speed = record['speed'][i]
        
#         if speed > 0:
#             record['speed'][0] = speed
#             break        
        
    return record

### Import Data

In [6]:
# sample code from https://sites.google.com/eng.ucsd.edu/fitrec-project/home

# path = Path("data/")
# out_path = str(path / "processed_endomondoHR_proper.npy")
# data = np.load(out_path)[0]

In [7]:
# sample code from https://sites.google.com/eng.ucsd.edu/fitrec-project/home

# data = []
# #with gzip.open('endomondoHR.json.gz') as f:
# with open('endomondoHR_proper.json') as f:
#     for l in f:
#         data.append(eval(l))

In [8]:
# old method

#file = "processed_endomondoHR_proper.npy"
# file = "processed_endomondoHR_proper_interpolate.npy"

# path = input_path + folder + file

#processed_endomondoHR_proper = np.load(path,mmap_mode='r')[0]
#data = np.memmap(path,mode='r+')

In [9]:
# old method

# file = "endomondoHR_proper.json"
# path = input_path + file

# num_workout = 300000
# data = {}
# index = 1

# with open(path) as f:    

#     for l in tqdm(f):
        
#         if index > num_workout:
#             break     

#         data[index] = eval(l)        
#         index += 1

### Output Data

In [10]:
# old method

# file = "endomondoHR_proper_sample.json"
# path = clean_path + folder + file

# with open(path, 'w') as f:
#     json.dump(data, f)

### Import, Split and Output File

In [11]:
# select json file for splitting

file_index = 3

input_filepath = [
            "endomondoHR.json",
            "endomondoMeta.json",
            "endomondoHR_proper.json", # next file is the same
            "endomondoHR_proper.json", # previous file is the same
            "processed_endomondoHR_proper.npy",
            "processed_endomondoHR_proper_interpolate.npy"        
            ]


output_filepath = [
            "endomondoHR/endomondoHR_",
            "endomondoMeta/endomondoMeta_",    
            "endomondoHR_proper/endomondoHR_proper_",                   # input file is endomondoHR_proper.json
            "endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_", # input file is endomondoHR_proper.json    
            "processed_endomondoHR_proper_",
            "processed_endomondoHR_proper_interpolate_"        
            ]

# init filepaths

in_path = raw_path + input_filepath[file_index]
out_path = input_path + output_filepath[file_index]

print(in_path)
print(out_path)

../data/0_raw/fitrec/endomondoHR_proper.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_


In [12]:
# import large json file and split into smaller json files (1000 jsons each)

json_per_file = 100
#json_per_file = 1000

if '.json' in in_path:

    data = {}
    index = 1
    file_no = 1
    
    #print('file_no:',file_no)

    with open(in_path) as f:

        for l in f:

            data[index] = eval(l) 
                
            if 'speed' not in data[index].keys():
                          
                data[index]['distance'] = [0] * 500
                data[index]['time_diff'] = [0] * 500
                data[index]['speed'] = [0] * 500
                
                data[index] = compute_distance(data[index],index)
                data[index] = compute_speed(data[index],index)

            # each file will contain n jsons
            if index % json_per_file == 0:            
                output_json(data,file_no)

                data = {}                
                file_no += 1
                
                #print('file_no:',file_no)

            index += 1

        # last file contains less than n jsons
        if index % json_per_file != 0:        
            output_json(data,file_no)            

../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0001.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0002.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0003.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0004.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0005.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0006.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0007.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0008.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0009.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0010.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0011.json
../data/1_input/fitre

../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0093.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0094.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0095.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0096.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0097.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0098.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0099.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0100.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0101.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0102.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0103.json
../data/1_input/fitre

../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0185.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0186.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0187.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0188.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0189.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0190.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0191.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0192.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0193.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0194.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0195.json
../data/1_input/fitre

../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0277.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0278.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0279.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0280.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0281.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0282.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0283.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0284.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0285.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0286.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0287.json
../data/1_input/fitre

../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0369.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0370.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0371.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0372.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0373.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0374.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0375.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0376.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0377.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0378.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_0379.json
../data/1_input/fitre

...

../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1289.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1290.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1291.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1292.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1293.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1294.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1295.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1296.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1297.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1298.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1299.json
../data/1_input/fitre

../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1381.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1382.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1383.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1384.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1385.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1386.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1387.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1388.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1389.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1390.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1391.json
../data/1_input/fitre

../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1473.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1474.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1475.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1476.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1477.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1478.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1479.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1480.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1481.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1482.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1483.json
../data/1_input/fitre

../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1565.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1566.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1567.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1568.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1569.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1570.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1571.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1572.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1573.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1574.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1575.json
../data/1_input/fitre

../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1657.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1658.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1659.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1660.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1661.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1662.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1663.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1664.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1665.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1666.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_1667.json
../data/1_input/fitre

In [13]:
if '.npy' in in_path:
    
    #data = np.load(in_path,mmap_mode='r')
    #data = np.memmap(in_path,mode='r+')
    
    #data = np.memmap(in_path,mode='r+')
    data = np.memmap(in_path,mode='r+',dtype='uint8')